Notebook to help with dashboard development

In [29]:
# set path to app.py
import sys
sys.path.append('../axle/app/src')

from config import settings
import pandas as pd
from dateutil import parser

DATA_PATH = settings.DATA_PATH

Read in data for config

In [37]:
user_archetypes = pd.read_csv(DATA_PATH.joinpath("user-archetypes.csv"))
time_cols = ['ave_plug-in_time', 'ave_plug-out_time']
perc_cols = ['ave_target_SoC', 'ave_plug-in_SoC']

for col in time_cols:
    user_archetypes[col] = user_archetypes[col].apply(lambda x: parser.parse(x).time())

# from '10%' string to 0.1
for col in perc_cols:
    user_archetypes[col] = user_archetypes[col].apply(lambda x: float(x.strip('%'))/100)

user_archetypes

,name,proportion,ave_miles_per_yr,battery_ave_kWh,ave_efficiency_mi_per_kWh,ave_plug-in_frequency_per_day,charger_kW,ave_plug-in_time,ave_plug-out_time,ave_target_SoC,ave_plug-in_SoC
0,Average (UK),0.40,9435,60.0,3.5,1.0,7,18:00:00,07:00:00,0.8,0.68
1,Intelligent Octopus average,0.30,28105,72.5,3.5,1.0,7,18:00:00,07:00:00,0.8,0.52
2,Infrequent charging,0.10,9435,60.0,3.5,0.2,7,18:00:00,07:00:00,0.8,0.18
3,Infrequent driving,0.10,5700,60.0,3.5,1.0,7,18:00:00,07:00:00,0.8,0.73
4,Scheduled charging,0.09,9435,60.0,3.5,1.0,7,22:00:00,09:00:00,0.8,0.68
5,Always plugged-in,0.01,9435,60.0,3.5,1.0,7,00:00:00,23:59:00,0.8,0.68


In [25]:
user_archetypes

,name,proportion,ave_miles_per_yr,battery_ave_kWh,ave_efficiency_mi_per_kWh,ave_plug-in_frequency_per_day,charger_kW,ave_plug-in_time,ave_plug-out_time,ave_target_SoC,ave_plug-in_SoC
0,Average (UK),0.40,9435,60.0,3.5,1.0,7,6:00 pm,7:00 am,80%,68%
1,Intelligent Octopus average,0.30,28105,72.5,3.5,1.0,7,6:00 pm,7:00 am,80%,52%
2,Infrequent charging,0.10,9435,60.0,3.5,0.2,7,6:00 pm,7:00 am,80%,18%
3,Infrequent driving,0.10,5700,60.0,3.5,1.0,7,6:00 pm,7:00 am,80%,73%
4,Scheduled charging,0.09,9435,60.0,3.5,1.0,7,10:00 pm,9:00 am,80%,68%
5,Always plugged-in,0.01,9435,60.0,3.5,1.0,7,12:00 am,11:59 pm,80%,68%


In [35]:
# convert to datetime
examples_time_string = '6:00 pm'
frmt = 'H:mm a'
examples_time = parser.parse(examples_time_string).time()
examples_time


datetime.time(18, 0)

In [36]:
from datetime import datetime
datetime.strptime(examples_time_string, frmt)

ValueError: time data '6:00 pm' does not match format 'H:mm a'

In [28]:
user_archetypes_display_name_map = {
    'name': 'Archetype',
    'proportion': 'Proportion',
    'ave_miles_per_yr': 'Average Miles Per Year',
    'battery_ave_kWh': 'Average Battery Size (kWh)',
    'ave_efficiency_mi_per_kWh': 'Average Efficiency (mi/kWh)',
    'ave_plug-in_frequency_per_day': 'Average Plug-in Frequency (per day)',
    'charger_kW': 'Charger Power (kW)',
    'ave_plug-in_time': 'Average Plug-in Time',
    'ave_plug-out_time': 'Average Plug-out Time',
    'ave_target_SoC': 'Average Target SoC',
    'ave_plug-in_SoC': 'Average Plug-in SoC'
}
